In [1]:
import numpy as np
import pandas as pd
from keras import layers
from keras import models
from keras import callbacks
from keras import backend as K
from keras.datasets import mnist
from matplotlib import pyplot as plt
from matplotlib import cm
from PIL import Image
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score,log_loss, mean_absolute_error,median_absolute_error
%matplotlib inline

Using TensorFlow backend.


In [2]:
from tqdm import tqdm
import pickle
def makefile(what,filename):
    with open(filename,"wb") as f3:
        pickle.dump(what,f3)

def readfile(filename):
    with open(filename,"rb") as f4:
        ans=pickle.load(f4)
    return ans

In [3]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import balanced_accuracy_score

In [4]:
images=["Lenna.png"]

In [5]:
from keras.backend import clear_session

In [6]:
import numpy as np
import random
import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf

In [7]:
dog=readfile("dog.pkl")

In [8]:
cat=readfile("cat.pkl")

In [9]:
y2=Image.open("Lenna.png").resize([64,64])

In [10]:
dog_train,dog_test=train_test_split(dog,test_size=0.2,random_state=0)
cat_train,cat_test=train_test_split(cat,test_size=0.2,random_state=0)

In [35]:
np.random.seed(seed=1)

os.environ['PYTHONHASHSEED'] = '0'
random.seed(0)

session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1)
#config.gpu_options.allow_growth = True
#tf.keras.backend.set_session(tf.Session(config=config))
tf.random.set_seed(0)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)
degree=[0,90,180,270]
values=[0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4]
convo=[13]
for goal in images:
    y2=Image.open(goal).resize([64,64])
    #y2=Image.open(goal).resize([32,32])
    data2=np.concatenate([dog_train,cat_train])
    #label2=cifar10.load_data()[0][1]
    label2=np.concatenate([np.zeros(10000),np.ones(10000)])
    #label2=label2.reshape(len(label2))
    x_test=np.concatenate([dog_test,cat_test])
    y_test=np.concatenate([np.zeros(2500),np.ones(2500)])
    data3=np.zeros([len(data2),64,64,3])
    for n in range(len(data2)):
        data3[n]=Image.fromarray(data2[n].astype("uint8"))
    data2=data3
    
    outputs=[]
    outputs4=[]
    outputs5=[]
    for c in range(len(convo)):
        outputs2=[]
        outputs3=[]
        for e in range(5):
            ensemble=[]
            for rot in degree:
                error_record=[]
                clear_session()
                unseen_score=np.zeros([2,9])
                AUC_score=np.zeros(2)
                for i in range(2):
                    clear_session()
                    x_train=data2[label2==i]
                    y_train=np.zeros([len(x_train),64,64,3])
                    for n in range(len(x_train)):
                        y_train[n]=y2.rotate(rot)
                    #x_train=x_train/255
                    y_train=y_train
                    #x_test=data2[label2!=i]
                    #x_test=x_test/255
                    x_test=x_test.reshape(len(x_test),64,64,3)
                    y_label=np.zeros(len(y_test))
                    y_label[y_test==i]=1
                    #(x_train, train), (x_test, y_test) = mnist.load_data()
                    image_height, image_width = 64, 64
                    # バックエンドに依存したチャネルの位置を調整する
                    if K.image_data_format() == 'channels_last':
                        x_train = x_train.reshape(x_train.shape[0],
                                                  image_height, image_width,3)
                        #x_test = x_test.reshape(x_test.shape[0],image_height, image_width, 1)
                        y_train = y_train.reshape(y_train.shape[0],
                                                  64, 64,3)
                        input_shape = (image_height, image_width,3)
                    else:
                        x_train = x_train.reshape(x_train.shape[0],
                                                  1, image_height, image_width)
                        x_test = x_test.reshape(x_test.shape[0],1, image_height, image_width)
                        input_shape = (1, image_height, image_width)

                    # Min-Max Normalization (0. ~ 1. の範囲に値を収める)
                    x_train = x_train.astype('float32')
                    y_train = y_train.astype('float32')
                    x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.3, random_state=e)
                    x_valid, x_valid2, y_valid, y_valid2 = train_test_split(x_valid, y_valid, test_size=0.3, random_state=e)
                    #x_test = x_test.astype('float32')
                    #x_train = (x_train - x_train.min()) / (x_train.max() - x_train.min())
                    #x_test = (x_test - x_test.min()) / (x_test.max() - x_test.min())
                    # 畳み込み演算を用いた AutoEncoder のネットワーク (Sequential API)
                    model = models.Sequential()
                    # 28 x 28 x 1
                    model.add(layers.Conv2D(32,kernel_size=(3,3),activation='relu',padding='same',input_shape=input_shape))
                    for n in range(convo[c]):
                        model.add(layers.Conv2D(32,kernel_size=(3,3),activation='relu',padding='same'))
                    model.add(layers.Conv2D(3, kernel_size=(3, 3),activation='relu', padding='same'))
                    # 28 x 28 x 1
                    model.compile(optimizer='adam',loss='mean_absolute_error')
                    # モデルの構造を確認する
                    fit_callbacks = [callbacks.EarlyStopping(monitor='val_loss',patience=5,mode='min')]
                    # モデルを学習させる
                    model.fit(x_train, y_train,epochs=100,batch_size=16,shuffle=True,validation_data=(x_valid, y_valid),callbacks=fit_callbacks)
                    result=model.predict(x_valid2)
                    result2=model.predict(x_test)
                    y_train=y_train.reshape(y_train.shape[0],4096,3)
                    result=result.reshape(result.shape[0],4096,3)
                    result2=result2.reshape(result2.shape[0],4096,3)
                    loss=np.zeros(len(result))
                    for n in range(len(result)):
                        loss[n]=mean_absolute_error(y_train[0],result[n])
                    loss_outlier=np.zeros(len(result2))
                    for n in range(len(result2)):
                        loss_outlier[n]=mean_absolute_error(y_train[0],result2[n])
                    error_record.append(loss_outlier)
                    #y_one=np.ones(len(loss))
                    #y_outlier=np.zeros(len(loss_outlier))
                    #data=np.concatenate([loss,loss_outlier])
                    #label=np.concatenate([y_one,y_outlier])
                    #plt.scatter(data,label)
                    #plt.hist([loss,loss_outlier],range=(0,0.3),stacked=False,bins=10)
                    m=0
                    AUC_score[i]=roc_auc_score(y_label,(-1)*loss_outlier)
                    for p in values:
                        y_pred=np.zeros(len(y_label))
                        threshold=int(len(loss)*p)+1
                        #print(loss[loss.argsort()[-threshold]])
                        y_pred[loss_outlier<=loss[loss.argsort()[-threshold]]]=1
                        #loss_outlier[loss_outlier>loss[loss.argsort()[-threshold]]
                        unseen_score[i][m]=balanced_accuracy_score(y_label,y_pred)
                        m+=1
                #print(unseen_score)
                #outputs.append(unseen_score)
                outputs2.append(AUC_score)
                outputs3.append(unseen_score)
                ensemble.append(error_record)
            outputs5.append(ensemble)
            outputs.append(outputs2)
            outputs4.append(outputs3)
    #makefile(outputs,"result_ocitn/cifar10_"+str(goal)+".pkl")

Train on 7000 samples, validate on 2100 samples
Epoch 1/100
7000/7000 [==============================] - 9s 1ms/step - loss: 72.6499 - val_loss: 28.9056
Epoch 2/100
7000/7000 [==============================] - 8s 1ms/step - loss: 26.4955 - val_loss: 24.8928
Epoch 3/100
7000/7000 [==============================] - 8s 1ms/step - loss: 24.5961 - val_loss: 24.1435
Epoch 4/100
7000/7000 [==============================] - 8s 1ms/step - loss: 24.0909 - val_loss: 23.8607
Epoch 5/100
7000/7000 [==============================] - 8s 1ms/step - loss: 23.8182 - val_loss: 23.9028
Epoch 6/100
7000/7000 [==============================] - 8s 1ms/step - loss: 23.6588 - val_loss: 23.5021
Epoch 7/100
7000/7000 [==============================] - 9s 1ms/step - loss: 23.4963 - val_loss: 23.4250
Epoch 8/100
7000/7000 [==============================] - 8s 1ms/step - loss: 23.3571 - val_loss: 23.3826
Epoch 9/100
7000/7000 [==============================] - 8s 1ms/step - loss: 23.1681 - val_loss: 23.0717
Epoch 1

7000/7000 [==============================] - 8s 1ms/step - loss: 22.6683 - val_loss: 23.0991
Epoch 24/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.6973 - val_loss: 22.5373
Epoch 25/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.6602 - val_loss: 23.6928
Epoch 26/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.5611 - val_loss: 23.0136
Epoch 27/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.5298 - val_loss: 22.4356
Epoch 28/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.5876 - val_loss: 23.4751
Epoch 29/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.4431 - val_loss: 22.4984
Epoch 30/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.4640 - val_loss: 22.4291
Epoch 31/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.4178 - val_loss: 22.5955
Epoch 32/100
7000/7000 [==============================] - 8

7000/7000 [==============================] - 8s 1ms/step - loss: 22.9101 - val_loss: 22.9919
Epoch 15/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.8410 - val_loss: 22.9411
Epoch 16/100
7000/7000 [==============================] - 9s 1ms/step - loss: 22.8012 - val_loss: 22.6297
Epoch 17/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.7195 - val_loss: 23.1419
Epoch 18/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.6943 - val_loss: 22.6574
Epoch 19/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.5633 - val_loss: 22.9070
Epoch 20/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.4997 - val_loss: 22.5014
Epoch 21/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.4367 - val_loss: 23.2372
Epoch 22/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.5244 - val_loss: 22.4555
Epoch 23/100
7000/7000 [==============================] - 8

7000/7000 [==============================] - 8s 1ms/step - loss: 22.8781 - val_loss: 22.7497
Epoch 17/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.9095 - val_loss: 23.4243
Epoch 18/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.8409 - val_loss: 22.7532
Epoch 19/100
7000/7000 [==============================] - 9s 1ms/step - loss: 22.7920 - val_loss: 22.8195
Epoch 20/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.7856 - val_loss: 22.6929
Epoch 21/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.6818 - val_loss: 22.5275
Epoch 22/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.6732 - val_loss: 23.6957
Epoch 23/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.6212 - val_loss: 22.8091
Epoch 24/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.5835 - val_loss: 22.4205
Epoch 25/100
7000/7000 [==============================] - 8

7000/7000 [==============================] - 8s 1ms/step - loss: 22.9017 - val_loss: 22.8514
Epoch 14/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.7982 - val_loss: 23.5660
Epoch 15/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.7031 - val_loss: 22.8029
Epoch 16/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.6852 - val_loss: 22.6679
Epoch 17/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.6521 - val_loss: 22.6013
Epoch 18/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.5608 - val_loss: 22.5617
Epoch 19/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.5133 - val_loss: 22.6449
Epoch 20/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.5061 - val_loss: 22.3772
Epoch 21/100
7000/7000 [==============================] - 9s 1ms/step - loss: 22.4610 - val_loss: 22.6965
Epoch 22/100
7000/7000 [==============================] - 8

...

7000/7000 [==============================] - 8s 1ms/step - loss: 22.7824 - val_loss: 22.7309
Epoch 19/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.6916 - val_loss: 23.1935
Epoch 20/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.6107 - val_loss: 22.6329
Epoch 21/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.5739 - val_loss: 22.8457
Epoch 22/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.6842 - val_loss: 22.7623
Epoch 23/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.4064 - val_loss: 22.7647
Epoch 24/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.3836 - val_loss: 22.3530
Epoch 25/100
7000/7000 [==============================] - 9s 1ms/step - loss: 22.3207 - val_loss: 22.2867
Epoch 26/100
7000/7000 [==============================] - 9s 1ms/step - loss: 22.3931 - val_loss: 22.3977
Epoch 27/100
7000/7000 [==============================] - 8

7000/7000 [==============================] - 8s 1ms/step - loss: 22.2099 - val_loss: 22.6782
Epoch 44/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.2504 - val_loss: 22.3006
Epoch 45/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.1833 - val_loss: 22.7205
Epoch 46/100
7000/7000 [==============================] - 9s 1ms/step - loss: 22.0864 - val_loss: 22.4231
Epoch 47/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.0895 - val_loss: 22.2724
Epoch 48/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.0941 - val_loss: 22.6005
Epoch 49/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.0256 - val_loss: 22.4180
Epoch 50/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.0629 - val_loss: 22.2880
Epoch 51/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.0008 - val_loss: 22.6479
Epoch 52/100
7000/7000 [==============================] - 8

7000/7000 [==============================] - 8s 1ms/step - loss: 22.1497 - val_loss: 22.0775
Epoch 35/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.1071 - val_loss: 22.1255
Epoch 36/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.0555 - val_loss: 22.8733
Epoch 37/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.0015 - val_loss: 22.3079
Epoch 38/100
7000/7000 [==============================] - 8s 1ms/step - loss: 21.9094 - val_loss: 22.0904
Epoch 39/100
7000/7000 [==============================] - 8s 1ms/step - loss: 21.9591 - val_loss: 22.0261
Epoch 40/100
7000/7000 [==============================] - 8s 1ms/step - loss: 21.8385 - val_loss: 22.0720
Epoch 41/100
7000/7000 [==============================] - 8s 1ms/step - loss: 21.8270 - val_loss: 21.8962
Epoch 42/100
7000/7000 [==============================] - 8s 1ms/step - loss: 21.8507 - val_loss: 21.8736
Epoch 43/100
7000/7000 [==============================] - 8

7000/7000 [==============================] - 8s 1ms/step - loss: 24.2081 - val_loss: 23.9036
Epoch 6/100
7000/7000 [==============================] - 8s 1ms/step - loss: 24.0077 - val_loss: 24.2111
Epoch 7/100
7000/7000 [==============================] - 8s 1ms/step - loss: 23.8720 - val_loss: 23.5959
Epoch 8/100
7000/7000 [==============================] - 8s 1ms/step - loss: 23.7858 - val_loss: 23.7549
Epoch 9/100
7000/7000 [==============================] - 8s 1ms/step - loss: 23.7381 - val_loss: 23.5301
Epoch 10/100
7000/7000 [==============================] - 8s 1ms/step - loss: 23.6295 - val_loss: 23.4758
Epoch 11/100
7000/7000 [==============================] - 8s 1ms/step - loss: 23.6396 - val_loss: 23.4652
Epoch 12/100
7000/7000 [==============================] - 8s 1ms/step - loss: 23.4858 - val_loss: 23.3950
Epoch 13/100
7000/7000 [==============================] - 8s 1ms/step - loss: 23.4273 - val_loss: 23.5134
Epoch 14/100
7000/7000 [==============================] - 8s 1m

7000/7000 [==============================] - 10s 1ms/step - loss: 22.0554 - val_loss: 22.1371
Epoch 32/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.0717 - val_loss: 22.2066
Epoch 33/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.0051 - val_loss: 23.2260
Epoch 34/100
7000/7000 [==============================] - 8s 1ms/step - loss: 22.0274 - val_loss: 22.3230
Epoch 35/100
7000/7000 [==============================] - 8s 1ms/step - loss: 21.9267 - val_loss: 22.4376
Epoch 36/100
7000/7000 [==============================] - 8s 1ms/step - loss: 21.9639 - val_loss: 22.1706


In [36]:
outputs6=[]
for ensemble in outputs5:
    CEtest2=None
    value = 0
    for scores in ensemble:
        if type(CEtest2)==type(None):
            CEtest2=np.array(scores)
        else:
            print("OK")
            CEtest2=CEtest2+np.array(scores)
    AUC_ensemble=np.zeros(2)
    for i in range(2):
        y_label=np.zeros(len(y_test))
        y_label[y_test==i]=1
        AUC_ensemble[i]=roc_auc_score(y_label,(-1)*CEtest2[i])
    outputs6.append(AUC_ensemble)

OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK


In [39]:
AUC_score2=[]
for ensemble in outputs5:
    AUC_score=[]
    for scores in ensemble:
        AUC=np.zeros(2)
        for i in range(2):
            y_label=np.zeros(len(y_test))
            y_label[y_test==i]=1
            AUC[i]=roc_auc_score(y_label,(-1)*scores[i])
        AUC_score.append(AUC)
    AUC_score2.append(AUC_score)

In [44]:
outputs7=[]
outputs7.append(np.array(AUC_score2).mean(axis=0))
outputs7.append(np.array(AUC_score2).std(axis=0))
outputs7.append(np.array(outputs6).mean(axis=0))
outputs7.append(np.array(outputs6).std(axis=0))

In [45]:
outputs7

[array([[0.63802395, 0.60664307],
        [0.6226727 , 0.6425989 ],
        [0.62702827, 0.60090451],
        [0.63226917, 0.61080888]]),
 array([[0.01047866, 0.01254681],
        [0.02029557, 0.01073282],
        [0.0089103 , 0.02883052],
        [0.00079365, 0.01375333]]),
 array([0.6726319 , 0.65783594]),
 array([0.00814839, 0.00925279])]

In [34]:
makefile(outputs7,"ITNensemble/cvd2.pkl")